# Study Note - Building AI Solutions with Azure Machine Learning
This notebook collects the notes taken through the course of **[Build AI solutions with Azure Machine Learning](https://docs.microsoft.com/en-us/learn/paths/build-ai-solutions-with-azure-ml-service/)** offered by Microsoft, with supplements from the **[documentation of Azure Machine Learning SDK for Python](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py)**.

This notebook contains Labs 06 - 07 of the learning course, which correspond to "Deploy and Consume Models" section in the exam guideline.

## 06 Deploy real-time machine learning services with Azure Machine Learning
In machine learning, *inferencing* refers to the use of a trained model to predict labels for new data on which the model has not been trained. In Azure Machine learning, you can create **real-time inferencing solutions by deploying a model as a service**, hosted in a **containerized platform** such as **Azure Kubernetes Services (AKS)**.

**Notes:** 
- **We can also deploy the model on Azure Container Instances (ACI) Web Service or local Docker-based service during development and testing.**
- **ACI web service is best for small scale testing and quick deployments, and AKS is for deloyments as a production-scale web service.**

To deploy a model as a real-time inferencing service, you must perform the following tasks:
1.	Register a trained model
2.	Define an inference configuration
    1.	Create an **entry script**: The entry script receives data submitted to a deployed web service and passes it to the model. It then takes the response returned by the model and returns that to the client. *The script is specific to your model.* It must understand the data that the model expects and returns.
        1.	`init()`: Called when the service is initialized - Typically, this function loads the model into a global object. This function is run only once, when the Docker container for your web service is started.
        2.	`run(inpute_data)`: Called with new data is submitted to the service - This function uses the model to predict a value based on the input data. Inputs and outputs of the run typically use JSON for serialization and deserialization. You can also work with raw binary data. You can transform the data before sending it to the model or before returning it to the client.

            ```python
            [To include entry script codes]
            ```
        
    2.	Create an environment
    3.	Combine the script and environment in an InferenceConfig

    ```python
    from azureml.core.model import InferenceConfig

    classifier_inference_config = InferenceConfig(runtime= "python",
                                                  source_directory = 'service_files',
                                                  entry_script="score.py",
                                                  conda_file="env.yml")
    ```

3.	Define a deployment configuration on the chosen compute target
    - AksCompute
    ```python
    from azureml.core.compute import ComputeTarget, AksCompute
    from azureml.core.webservice import AksWebservice
    ```

    - ACI deployment
    ```python
    from azureml.core.webservice import AciWebservice
    ```
    
    - local Docker-based service
    ```python
    from azureml.core.webservice import LocalWebservice
    ```
4.	Deploy the model
```python
service = Model.deploy(workspace=ws,
                       name = 'classifier-service',
                       models = [model], #1. Model registered
                       inference_config = classifier_inference_config, # 2. Inference Configuration
                       deployment_config = classifier_deploy_config, # 3. deployment configuration
                       deployment_target = production_cluster) # (Optional) 3. deployment configuration
service.wait_for_deployment(show_output = True)
print(service.state)
```

To delete a deployed web service, use `service.delete()`. To delete a registered model, use `model.delete()`.

#### [Additional Topic: Create an endpoint](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-azure-kubernetes-service#create-an-endpoint)

**To create an endpoint, use `AksEndpoint.deploy_configuration` instead of `AksWebservice.deploy_configuration()`.**

```python
import azureml.core,
from azureml.core.webservice import AksEndpoint
from azureml.core.compute import AksCompute
from azureml.core.compute import ComputeTarget
# select a created compute
compute = ComputeTarget(ws, 'myaks')
namespace_name= endpointnamespace
# define the endpoint and version name
endpoint_name = "mynewendpoint"
version_name= "versiona"
# create the deployment config and define the scoring traffic percentile for the first deployment
endpoint_deployment_config = AksEndpoint.deploy_configuration(cpu_cores = 0.1, memory_gb = 0.2,
                                                              enable_app_insights = True,
                                                              tags = {'sckitlearn':'demo'},
                                                              description = "testing versions",
                                                              version_name = version_name,
                                                              traffic_percentile = 20)
 # deploy the model and endpoint
 endpoint = Model.deploy(ws, endpoint_name, [model], inference_config, endpoint_deployment_config, compute)
 # Wait for he process to complete
 endpoint.wait_for_deployment(True)
```

To *consume* a deployed real-time service (or model or endpoint), we’ll need the following: **(Note: Recall the consume tab in AML Studio.)**
-	HTTP Post/ Url **(Note: Recall the step to copy the REST url on AML studio and paste it in the script.)**
-	Key **(Note: Recall the step to copy the primary key on AML studio and paste it in the script.)**

## 07 Deploy batch inference pipelines with Azure Machine Learning
The steps are not very consistent between lectures and lab codes. Refer to the lab codes when there’s inconsistency.
1.	Register a model
2.	Create a scoring script and define a run context that includes the dependencies required by the script
3.	Create a pipeline with **ParallelRunStep** (As the chapter name suggests, we’re going to create a step in pipeline)
```python
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
# Define the parallel run step step configuration
# Create the parallel run step
```
4.	Run the pipeline and retrieve the step output
